## Avinash Kappa - Assignment 4b
### Tried several times with Colab and the training got interrupted couple of times. 
### In the last trial, I kept saving the weights everytime there is an improvement in the validation accuracy.  
### Was able to reach 42 epochs before the runtime got terminated. Must be because of 12 hour Colab limit. 
### The training accuracy as of the last reached epoch was 0.8540
### I loaded the saved weights and did the evaluation. 

In [0]:
# https://keras.io/
!pip install -q keras
import keras

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
import tensorflow as tf
from keras import backend as k
from keras.callbacks import ModelCheckpoint

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 50
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 24
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


## Saving the weights during the training

In [0]:
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [12]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 24)   648         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 24)   96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 24)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])


## Load the saved weights from the training

In [0]:
model.load_weights("weights.best.hdf5")

In [0]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    callbacks=callbacks_list,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
43776/50000 [=========================>....] - ETA: 41s - loss: 1.5247 - acc: 0.4364

50000/50000 [==============================] - 352s 7ms/step - loss: 1.4827 - acc: 0.4520 - val_loss: 1.4029 - val_acc: 0.5155

Epoch 00001: val_acc improved from -inf to 0.51550, saving model to weights.best.hdf5
Epoch 2/50
18816/50000 [==========>...................] - ETA: 3:21 - loss: 1.1426 - acc: 0.5894

50000/50000 [==============================] - 340s 7ms/step - loss: 1.0830 - acc: 0.6094 - val_loss: 1.3846 - val_acc: 0.5514

Epoch 00002: val_acc improved from 0.51550 to 0.55140, saving model to weights.best.hdf5
Epoch 3/50
11840/50000 [======>.......................] - ETA: 4:08 - loss: 0.9638 - acc: 0.6555

50000/50000 [==============================] - 341s 7ms/step - loss: 0.9357 - acc: 0.6646 - val_loss: 1.5078 - val_acc: 0.5555

Epoch 00003: val_acc improved from 0.55140 to 0.55550, saving model to weights.best.hdf5
Epoch 4/50
 9984/50000 [====>.........................] - ETA: 4:18 - loss: 0.8572 - acc: 0.6962

50000/50000 [==============================] - 341s 7ms/step - loss: 0.8398 - acc: 0.7026 - val_loss: 1.0534 - val_acc: 0.6676

Epoch 00004: val_acc improved from 0.55550 to 0.66760, saving model to weights.best.hdf5
Epoch 5/50
 9472/50000 [====>.........................] - ETA: 4:22 - loss: 0.7806 - acc: 0.7199

...

50000/50000 [==============================] - 341s 7ms/step - loss: 0.2555 - acc: 0.9087 - val_loss: 0.5951 - val_acc: 0.8436

Epoch 00037: val_acc did not improve from 0.85840
Epoch 38/50
11072/50000 [=====>........................] - ETA: 4:13 - loss: 0.2390 - acc: 0.9141

50000/50000 [==============================] - 341s 7ms/step - loss: 0.2515 - acc: 0.9098 - val_loss: 0.6072 - val_acc: 0.8424

Epoch 00038: val_acc did not improve from 0.85840
Epoch 39/50
11136/50000 [=====>........................] - ETA: 4:12 - loss: 0.2438 - acc: 0.9134

50000/50000 [==============================] - 342s 7ms/step - loss: 0.2502 - acc: 0.9111 - val_loss: 0.6137 - val_acc: 0.8377

Epoch 00039: val_acc did not improve from 0.85840
Epoch 40/50
11200/50000 [=====>........................] - ETA: 4:11 - loss: 0.2351 - acc: 0.9184

50000/50000 [==============================] - 341s 7ms/step - loss: 0.2428 - acc: 0.9136 - val_loss: 0.8074 - val_acc: 0.8000

Epoch 00040: val_acc did not improve from 0.85840
Epoch 41/50
11200/50000 [=====>........................] - ETA: 4:10 - loss: 0.2276 - acc: 0.9148

50000/50000 [==============================] - 341s 7ms/step - loss: 0.2368 - acc: 0.9151 - val_loss: 0.5809 - val_acc: 0.8540

Epoch 00041: val_acc did not improve from 0.85840
Epoch 42/50
 4352/50000 [=>............................] - ETA: 4:57 - loss: 0.2160 - acc: 0.9226

In [14]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 23s 2ms/step
Test loss: 0.5102088269948959
Test accuracy: 0.8584


In [0]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')